## Ensiksi syötetään alkutiedot 

Alkutietojen muuttujat:
- bp = bruttopalkka (€/vuosi)
- ika = Veronmaksajan ikä
- k_k = kuulun kirkkoon (True/False)
- t_m = Työttömyyskassan jäsenmaksu (€/vuosi)
- mk = Matka kulut per vuosi
- mko = Matkakulujen omavastuu per vuosi
- k_kunta = Kotikunta verotusvuonna

Seuraan tässä laskennassa veronmaksajien esimerkkiä --> https://www.veronmaksajat.fi/Palkka-ja-elake/Jarin-palkka/

In [1]:
bp = 40000
k_k = True
t_m = 0
mk = 0
mko = 750
ika = 25

## Puhdas ansiotulo

Ensiksi lasketaan verotuksessa puhdas ansiotulo (pat). Tämä saadaan vähentämällä luonnolliset vähennykset bruttopalkasta. Ansiotulosta vähennetään aina tulonhankkimisvähennys (thv)

In [2]:
thv = 750

# Lasketaan ylittääkö matkakustannukset omavastuun.
if mk < mko:
    mtv = 0
else:
    mtv = mk - mko

pat = bp - t_m - mk - thv

print('Puhdas ansiotulo on: {}€'.format(pat))

Puhdas ansiotulo on: 39250€


## Sosiaalivakuutusmaksut ja sairausvakuutusmaksu

Tämän jälkeen vähennetään työeläkemaksu

In [3]:
if ika >= 17 and ika <= 52:
    telm_p = 0.0715
elif ika >= 53 and ika <= 62:
    telm_p = 0.0865
elif ika >= 63 and 67 <= 62:
    telm_p = 0.0715
else:
    telm_p = 0

tyel_m = pat*telm_p
print('TyEL maksu: {:.1f}€'.format(tyel_m))

TyEL maksu: 2806.4€


Seuraavaksi lasketaan työttömyysvakuutusmaksu, jos työntekijä on 17 - 64 -vuotias

In [4]:
if ika >= 17 and ika <= 64:
    tvm_p = 0.014
else:
    tvm_p = 0.0

tvm = pat*tvm_p
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(tvm))

Työttömyysvakuutusmaksu maksu: 549.5€


Seuraavaksi lasketaan sairausvakuutuksen päivärahamaksu. Vuositulot ovat alle 14766€. Maksua ei peritä

In [5]:
if ika >= 16 and ika <= 67 and pat >= 14766:
    prm_p = 0.0136
else:
    prm_p = 0


svm = pat*prm_p
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(svm))

Työttömyysvakuutusmaksu maksu: 533.8€


Sitten lasketaan sairaanhoitomaksu palkkatulosta (0,68%)

In [6]:
shm = pat*0.0068
print('Työttömyysvakuutusmaksu maksu: {:.1f}€'.format(shm))

Työttömyysvakuutusmaksu maksu: 266.9€


## Verotettava tulo sosiaalivakuutusmaksujen ja sairausvakuutusmaksun jälkeen

In [7]:
v_t = pat - shm - svm - tvm - tyel_m
print('Verotettavat tulo: {:.1f}€'.format(v_t))

Verotettavat tulo: 35093.4€


## Kunnallisvero

Sitten lasketaan kunnallisverot. Ensin aloitetaan kunnallisverotuksen asiotulovähennykestä (atv). Kunnallisveron ansiotulovähennys lasketaan ureasta eri tuloista:
- Veron alaisesta palkkatulosta
- Muusta toiselle suoritetusta työstä, tehtävästä tai palveluksesta saadut ansiotulot
- Ansiotulosta pidätettävistä käyttökorvauksista
- Ansiotulona verotettavasta  osingosta
- Jaettavasta yritystulosta, joka on asiotulon-osuutta
- Uhtymän osakkaan elinkeinotoiminnan tai maatalouden ansiotulo-osuuden osuudesta

Tässä esimerkissä kuitenkin keskitytään vain veron alaiseen palkkatuloon

In [8]:
bp = 40000

if bp <= 2500:
    atv = 0
elif bp > 2500 and bp  <= 7230:
    atv = (bt - 2500) * 0.51
elif bp  > 7230: 
    atv = (7230 - 2500) * 0.51 + (bp - 7230) * 0.28

if atv > 3570: # Asiotulovähennyksen maksimimäärä on 3570
    atv = 3570

# Jos kuitenkin verovelvollisen puhtaat ansiotulot ylittävät 14000€ vähennyksen määrä pienenee puhtaan ansiotulon ylittävältä osalta.

if pat > 14000:
    atv = atv - (pat - 14000) * 0.045

print('Asiotulovähennys kunnallisverotuksesta: {:.2f}€'.format(atv))


Asiotulovähennys kunnallisverotuksesta: 2433.75€


## Kunnallisverotuksen perusvähennys

Perusvähennys kunnallisverotuksessa on maksimissaan 3305€. Tähän vaikuttaa myös muut vähennykset tulojen lisäksi. Puhtaasta ansiotulosta vähennetään sosiaalivakuutusmaksut, kunnallisverotuksen ansiotulovähennys sekä muut mahdolliset kunnallisverotuksen vähennykset. 

In [9]:

kvp = 3305 # Perusvähennyksen maksimiarvo

kvp_va = v_t - atv
if kvp_va > kvp:
    kvp = kvp - kvp_va*0.18

if kvp < 0:
    kvp = 0

print('Kunnallisverotuksen perusvähennys: {:.2f}€'.format(kvp))



Kunnallisverotuksen perusvähennys: 0.00€


## Kunnallisverotuksessa verotettava tulo

In [10]:
print(v_t - atv - kvp)

32659.674999999996


In [11]:
import requests
from io import BytesIO
from pdfminer.high_level import extract_text
import numpy as np
import pandas as pd

url = 'https://www.vero.fi/contentassets/be5e703ff5e34099a2ea2f74a33eaa68/kuntien-ja-seurakuntien-tuloveroprosentit-vuonna-2021.pdf'

rq = requests.get(url)
text = extract_text(BytesIO(rq.content))

data = text.split('Kunta ')[1].split('Suomen saksalaisen')[0]

temp = data.split('\n\n')
kunnat = ''.join(temp[1::4]).replace('\x0c','\n').replace(' ','').split('\n')
kunnallis_veroprosentti  = ''.join(temp[2::4]).replace('\n','').replace(',','.').split(' ')
ev_lut_veroprosentti = ''.join(temp[3::4]).replace('\n','').replace(',','.').split(' ')
ort_veroprosentti = ''.join(temp[4::4]).replace('\n','').replace(',','.').split(' ')

df_kunnat = pd.DataFrame(data = kunnat, columns = ['Kunta']).replace('', np.nan).dropna()
df_kunnallis_veroprosentti = pd.DataFrame(data = kunnallis_veroprosentti, columns = ['Kunnan\ntuloveroprosentti']).replace('', np.nan).dropna()
df_ev_lut_veroprosentti = pd.DataFrame(data = ev_lut_veroprosentti, columns = ['Ev.lut.\nseurakunnan\ntuloveroprosentti']).replace('', np.nan).dropna()
df_ort_veroprosentti = pd.DataFrame(data = ev_lut_veroprosentti, columns = ['Ortodoks.\nseurakunnan\ntuloveroprosentti']).replace('', np.nan).dropna()

result = pd.concat([df_kunnat,df_kunnallis_veroprosentti, df_ev_lut_veroprosentti, df_ort_veroprosentti], axis = 1)

In [12]:
print(result.to_markdown())

|     | Kunta              |              Kunnan |             Ev.lut. |           Ortodoks. |
|     |                    |   tuloveroprosentti |         seurakunnan |         seurakunnan |
|     |                    |                     |   tuloveroprosentti |   tuloveroprosentti |
|----:|:-------------------|--------------------:|--------------------:|--------------------:|
|   0 | Akaa               |               22.25 |                1.7  |                1.7  |
|   1 | Alajärvi           |               21.75 |                1.75 |                1.75 |
|   2 | Alavieska          |               22    |                1.8  |                1.8  |
|   3 | Alavus             |               21.25 |                1.75 |                1.75 |
|   4 | Asikkala           |               20.75 |                1.75 |                1.75 |
|   5 | Askola             |               21.5  |                1.75 |                1.75 |
|   6 | Aura               |               21.5  |